In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplot
from collections import Counter
import seaborn as sns
import psycopg2

In [32]:
# sklearn processing and classification 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [33]:
# sklearn classification model evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [34]:
# import the data
file_path = pd.read_csv('C:/Users/lizzi/School/Group_5_Final_Project/violence_count_df.csv')


In [35]:
file_path.tail()

,ROW_ID,INCIDENT_REPORT_ID,DIVISION_ID,PLACE_TYPE_DESCRIPTION,CLEARANCE_STATUS,HIGHEST_NIBRS_CODE,CITY_NEW,VIOLENCE
489979,490718,20220513_2308_489980,12,Residential,Open,13A,Charlotte,violent
489980,490719,20220513_2308_489981,27,Residential,Open,290,Charlotte,non-violent
489981,490720,20220513_2308_489982,22,Commercial Place,Cleared by Arrest,13B,Charlotte,violent
489982,490721,20220513_2308_489983,2,Residential,Open,290,Charlotte,non-violent
489983,490722,20220513_2308_489984,16,Commercial Place,Cleared by Arrest,90C,Charlotte,non-violent


In [36]:
file_path.isnull().mean().sort_values()

ROW_ID                    0.0
INCIDENT_REPORT_ID        0.0
DIVISION_ID               0.0
PLACE_TYPE_DESCRIPTION    0.0
CLEARANCE_STATUS          0.0
HIGHEST_NIBRS_CODE        0.0
CITY_NEW                  0.0
VIOLENCE                  0.0
dtype: float64

In [37]:
# prepare data
# split into input and target features
X = file_path[['ROW_ID', 'DIVISION_ID']]
y = file_path.VIOLENCE

In [38]:
X.describe()

,ROW_ID,DIVISION_ID
count,489984.000000,489984.000000
mean,245359.852471,14.330948
std,141654.513862,8.494614
min,0.000000,0.000000
25%,122696.750000,7.000000
50%,245365.500000,14.000000
75%,368030.250000,21.000000
max,490722.000000,92.000000


In [39]:
# build model
# split into test and training sets
X_train, X_test, y_train, y_test = train_test_split(
X, y, random_state=1, stratify=y)


In [40]:
logisticRegr = LogisticRegression()


In [41]:
logisticRegr.fit(X_train, y_train)


LogisticRegression()

In [42]:
classifier = LogisticRegression(solver='lbfgs',
   max_iter=200,
   random_state=1)

In [43]:
classifier.fit(X_train, y_train)


LogisticRegression(max_iter=200, random_state=1)

In [44]:
y_pred = classifier.predict(X_test)


In [45]:
print(accuracy_score(y_test, y_pred))


0.5355276907001045


In [46]:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
   intercept_scaling=1, max_iter=200, multi_class='warn', penalty='12',
   random_state=1, solver='lbfgs', warm_start=False)

LogisticRegression(max_iter=200, multi_class='warn', penalty='12',
                   random_state=1)

In [47]:
# create predictions with results in Pandas DF
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
163078,non-violent,non-violent
393155,non-violent,non-violent
154781,non-violent,non-violent
357801,non-violent,non-violent
138804,non-violent,violent
...,...,...
278135,non-violent,violent
204450,non-violent,violent
486892,non-violent,non-violent
136791,non-violent,violent


In [48]:
# Evaluate test performance
accuracy_score(y_test, predictions)


0.5355276907001045

In [49]:
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[65600     0]
 [56896     0]]


In [55]:
# Evaluatinf on test set
test_acc = accuracy_score(y_test, y_pred)
print("The Accuracy for Test Set is {}".format(test_acc*100))

The Accuracy for Test Set is 53.552769070010456


In [24]:
# generating classification report
print(classification_report(y_test, y_pred))


C:\Users\lizzi\anaconda3\envs\PythonData\envs\PythonData\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lizzi\anaconda3\envs\PythonData\envs\PythonData\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

 non-violent       0.54      1.00      0.70     65600
     violent       0.00      0.00      0.00     56896

    accuracy                           0.54    122496
   macro avg       0.27      0.50      0.35    122496
weighted avg       0.29      0.54      0.37    122496



C:\Users\lizzi\anaconda3\envs\PythonData\envs\PythonData\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

C:\Users\lizzi\anaconda3\envs\PythonData\envs\PythonData\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                   pre       rec       spe        f1       geo       iba       sup

non-violent       0.54      1.00      0.00      0.70      0.00      0.00     65600
    violent       0.00      0.00      1.00      0.00      0.00      0.00     56896

avg / total       0.29      0.54      0.46      0.37      0.00      0.00    122496



In [ ]:
# Connect to your postgres DB
conn = psycopg2.connect("dbname=g5-final-project user=postgres password=policedata5")

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a query
cur.execute("SELECT * FROM file_path")

In [ ]:
# Retrieve query results
records = cur.fetchall()